<a href="https://colab.research.google.com/github/Aftabgazali/CNN_On_MNIST_DATASET/blob/main/CNN_On_MNIST_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the Libraries

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Setting up the Device Agnostic Code

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

# Preparing the Data, Applying transforms and converting to Tensors

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
train_data = datasets.MNIST(root="data", train= True, transform= transform, target_transform= None, download= True)
test_data = datasets.MNIST(root="data", train= False, transform= transform, target_transform= None, download= True)

In [ ]:
len(train_data), len(test_data)

# Visualizing the Data

In [ ]:
image, label = train_data[0]

## So the dataset is already in a grayscale form

In [ ]:
image.shape, label

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
figure = plt.figure(figsize=(10,7))
rows,cols = 4,4
for i in range(1, rows*cols+1):
  random_index = torch.randint(0, len(train_data), size=[1]).item()
  image, label = train_data[random_index]
  figure.add_subplot(rows,cols,i)
  plt.imshow(image.squeeze())
  plt.title(class_names[label])
  plt.axis(False)

# Preparing the Data into Batches

In [ ]:
BATCH_SIZE = 32
train_data_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_data_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
len(train_data_loader), len(test_data_loader)

# Building a Simple Linear Model

In [ ]:
class MNISTV0(nn.Module):
  def __init__(self, input_shape:int, output_shape:int, hidden_units:int):
    super().__init__()
    self.layer_stacked = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=hidden_units),
        nn.Linear(in_features=hidden_units, out_features=output_shape)
    )

  # Forward Pass
  def forward(self, x:torch.Tensor):
    return self.layer_stacked(x)

# Creating the Instance of the baseline model
model_v0 = MNISTV0(input_shape=28*28, output_shape=len(class_names), hidden_units=4).to(device)

In [ ]:
model_v0.state_dict

## Model Loss & Optimizer Functions

In [ ]:
model_loss = nn.CrossEntropyLoss()
model_optimizer = torch.optim.Adam(params = model_v0.parameters())

## Building the Accuracy Function

In [ ]:
def model_accuracy(y_true, y_pred):
  acc = torch.eq(y_true,y_pred).sum().item()
  return (acc/len(y_true))*100

## Building the Training & Testing Step

In [ ]:
from tqdm.auto import tqdm

def train_step(model: torch.nn.Module, no_of_epochs:int, data_loader: torch.utils.data, model_loss:torch.nn.Module, model_acc, model_optimizer:torch.optim, device: torch.device = device):
  """
    Training Step for Model
  """
  for epoch in tqdm(range(no_of_epochs)):
    print(f"Epochs {epoch} ------------------------- ")
    # Training Mode:
    model.train()
    train_loss, train_acc = 0,0
    for batch, (X,y) in enumerate(data_loader):
      X,y = X.to(device), y.to(device)
      # Forward Pass
      y_logits = model(X)

      # Calculate the Loss
      loss = model_loss(y_logits, y)
      train_loss += loss

      # Caculate the training acc
      train_acc += model_acc(y, y_logits.argmax(dim=1))
      # Optimizer zero grad
      model_optimizer.zero_grad()

      # Loss Backward
      loss.backward()

      # Optmizier step
      model_optimizer.step()

      if batch % 400 == 0:
        print(f"Looked Through {batch * len(X)} / {len(train_data_loader.dataset)} samples")

    # Update the training loss & training accuracy
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Training Loss {train_loss} Training Accuracy {train_acc}")

In [ ]:
train_step(model_v0, 3, train_data_loader, model_loss, model_accuracy, model_optimizer)

In [ ]:
from tqdm.auto import tqdm

def test_step(model: torch.nn.Module, data_loader: torch.utils.data, model_loss:torch.nn.Module, model_acc,device: torch.device = device):
  """
    Testing Step for the Model
  """
  # Testing Mode:
  model.eval()
  with torch.inference_mode():
    test_loss, test_acc = 0,0
    for X_test,y_test in tqdm(data_loader):
        # Forward Pass
        X_test, y_test = X_test.to(device), y_test.to(device)
        y_logits = model(X_test)

        # Calculate the Loss
        test_loss += model_loss(y_logits, y_test)

        # Caculate the training acc
        test_acc += model_acc(y_test, y_logits.argmax(dim=1))
      # Update the training loss & training accuracy
    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    print(f"Training Loss {test_loss:.2f} Training Accuracy {test_acc:.2f}")

In [ ]:
test_step(model_v0,test_data_loader, model_loss, model_accuracy)

## Building Eval Model Method

In [ ]:
def eval_model(model:torch.nn.Module, data_loader: torch.utils.data, model_loss:torch.nn.Module, model_acc,device: torch.device = device):
  # Testing Mode:
  model.eval()
  with torch.inference_mode():
    test_loss, test_acc = 0,0
    for X_test,y_test in tqdm(data_loader):
        X_test,y_test = X_test.to(device),y_test.to(device)
        # Forward Pass
        y_logits = model(X_test)

        # Calculate the Loss
        test_loss += model_loss(y_logits, y_test)

        # Caculate the training acc
        test_acc += model_acc(y_test, y_logits.argmax(dim=1))
      # Update the training loss & training accuracy
    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    return {'model':model.__class__.__name__,'Loss':(test_loss*100).item(),'Accuracy':test_acc}

In [ ]:
model_v0_results= eval_model(model_v0,test_data_loader, model_loss, model_accuracy)

# Building the Non-Linear Model

In [ ]:
class MNISTV1(nn.Module):
  def __init__(self, input_shape:int, output_shape:int, hidden_units:int):
    super().__init__()
    self.layer_stacked = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units, out_features=output_shape),
    )
  # Forward Pass
  def forward(self, x:torch.Tensor):
    return self.layer_stacked(x)

# Creating the Instance of the baseline model
model_v1 = MNISTV1(input_shape=28*28, output_shape=len(class_names), hidden_units=4).to(device)

## Creating Loss & Optimizer functions

In [ ]:
model_loss = nn.CrossEntropyLoss()
model_optimizer = torch.optim.Adam(params = model_v1.parameters())

## Building Training & Testing Step

In [ ]:
train_step(model_v1, 3, train_data_loader, model_loss, model_accuracy, model_optimizer)

In [ ]:
test_step(model_v1,test_data_loader, model_loss, model_accuracy)

# Building the CNN Model

In [ ]:
class MNISTV2(nn.Module):
  def __init__(self, input_shape:int, output_shape:int, hidden_units:int):
    super().__init__()
    self.conv_2d_layer_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=2,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units*4, kernel_size=2,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier_layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*4*15*15,out_features=output_shape)
    )
  # Forward Pass
  def forward(self, x:torch.Tensor):
    x = self.conv_2d_layer_1(x)
    #print(f"Shape of X {x.shape}")
    return self.classifier_layer(x)

# Creating the Instance of the baseline model
model_v2 = MNISTV2(input_shape=1, output_shape=len(class_names), hidden_units=16).to(device)

In [ ]:
test_image = torch.randn(size=(1,28,28))
model_v2(test_image.unsqueeze(0).to(device))

In [ ]:
model_loss = nn.CrossEntropyLoss()
model_optimizer = torch.optim.Adam(params = model_v2.parameters())

In [ ]:
train_step(model_v2, 5, train_data_loader, model_loss, model_accuracy, model_optimizer)

In [ ]:
test_step(model_v2,test_data_loader, model_loss, model_accuracy)

# Achieved an Impressive Accuracy of 99.57%

# Comparing Model Results

In [ ]:
model_v0_results= eval_model(model_v0,test_data_loader, model_loss, model_accuracy)
model_v1_results= eval_model(model_v1,test_data_loader, model_loss, model_accuracy)
model_v2_results= eval_model(model_v2,test_data_loader, model_loss, model_accuracy)

In [ ]:
model_results_df = pd.DataFrame([model_v0_results,model_v1_results,model_v2_results])
model_results_df

In [ ]:
model_results_df.set_index('model')['Accuracy'].plot(kind='barh', color='g')
plt.xlabel('Accuracy %')
plt.ylabel('Models')

# Making Predictions

In [ ]:
test_images_per_batch, test_labels_per_batch = next(iter(test_data_loader))

In [ ]:
figure = plt.figure(figsize=(16,8))
rows, cols = 4,4

for i in range(1, rows*cols+1):
  random_index = torch.randint(0, len(test_images_per_batch), size=[1]).item()
  image, label = test_images_per_batch[random_index], test_labels_per_batch[random_index]
  y_logits = model_v2(image.unsqueeze(0).to(device))
  test_prediction_label = y_logits.argmax(dim=1)
  figure.add_subplot(rows,cols,i)
  title = f"Predicted {class_names[test_prediction_label]} | Actual {class_names[label]}"
  plt.imshow(image.squeeze())
  if class_names[test_prediction_label] == class_names[label]:
    plt.title(title,color="g")
  else:
    plt.title(title, color="r")
  plt.axis(False)

# Making Prediction using Real Images

In [ ]:
from PIL import Image

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

# Load the image from an outside source
image = Image.open('test_3.png')

# Preprocess the image with blur and add a batch dimension
input_tensor = preprocess(image).unsqueeze(0)

In [ ]:
plt.imshow(input_tensor.squeeze())

In [ ]:
y_logitss = model_v2(input_tensor.to(device))
y_logitss

In [ ]:
pred = y_logitss.argmax(dim=1)
plt.imshow(input_tensor.squeeze())
plt.title(f"Predicted Label {class_names[pred]} | Actual Label is 3")
plt.axis(False)